In [ ]:
import numpy as np
import os 
import keras
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
#from livelossplot.keras import PlotLossesCallback
from os import path
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
from keras.models import Sequential, Model
from keras.layers import Input
from keras import initializers
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')

In [ ]:
X_tot=np.empty(shape=(1,9))
Y_tot=np.empty(shape=(1,4),dtype=np.int8)
for i in range(1,55):
    s = 'user_features/user'
    s+=str(i)+'_cb.txt'
    if(path.exists(s)):
        #print(s)
        dataset =  np.loadtxt(s,delimiter = " ")
        if dataset.ndim == 1:
            dataset.shape = (1,10)
        Y = dataset[:,9]
        trn = dataset[:,0:9]
        #X_res, y_res = sm.fit_resample(trn, Y)
        labels = []
        for l in Y:
            if l == 0:
                labels.append([1,0,0,0])
            elif l == 1 :
                labels.append([0,1,0,0])
            elif l == 2 :
                labels.append([0,0,1,0])
            else :
                labels.append([0,0,0,1])
        trn=np.array(trn)
        labels = np.array(labels)
        if i==5:
            print(labels.shape)
        X_tot=np.concatenate((X_tot,trn),axis=0)
        Y_tot=np.concatenate((Y_tot,labels),axis=0)
X_tot = sc.fit_transform(X_tot)       

In [ ]:
np.sum(Y_tot,axis=0)
#Y_tot.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tot, Y_tot, test_size = 0.2)

In [ ]:
feature_extractor = Sequential()
feature_extractor.add(Dense(16, activation='relu',input_dim=9))
feature_extractor.add(Dropout(0.2))
feature_extractor.add(Dense(32, activation='relu'))
feature_extractor.add(Dropout(0.15))
nd=32
feature_extractor.add(Dense(nd, activation='relu'))
feature_extractor.add(Dense(4, activation='softmax'))
adm=keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
feature_extractor.compile(loss='categorical_crossentropy',optimizer=adm,metrics=['accuracy'])
feature_extractor.summary()

In [ ]:
feature_extractor.fit(X_train,y_train,epochs=50,batch_size=8,validation_data=(X_train,y_train))

In [ ]:
mod1=feature_extractor
mod1.pop()

mod1.summary()

In [ ]:
y= feature_extractor.predict(X_test)
y.shape

In [ ]:
# feature_extractor.add(Dropout(0.15))
row = Input(shape = (9,))
features = feature_extractor(row)
User_classifiers = []
User_emotions = []
Models = []
Labels = []
Train = []
Labels_test = []
Train_test = []

In [ ]:
nd= 32
for i in range(1,55):
    X_tot=np.empty(shape=(1,9))
    Y_tot=np.empty(shape=(1,4),dtype=np.int8)
    s = 'user_features/user'
    s+=str(i)+'_cb.txt'
    if(path.exists(s)):
        print(s)
        dataset =  np.loadtxt(s,delimiter = " ")
        if dataset.ndim == 1:
            dataset.shape = (1,10)
        Y = dataset[:,9]
        trn = dataset[:,0:9]
        labels = []
        for l in Y:
            if l == 0:
                labels.append([1,0,0,0])
            elif l == 1 :
                labels.append([0,1,0,0])
            elif l == 2 :
                labels.append([0,0,1,0])
            else :
                labels.append([0,0,0,1])
        labels = np.array(labels)
        #X_tot=np.concatenate((X_tot,trn),axis=0)
        #Y_tot=np.concatenate((Y_tot,labels),axis=0)
        X_train, X_test, y_train, y_test = train_test_split(trn, labels, test_size=0.2)
        X_train=sc.transform(X_train)
        X_test=sc.transform(X_test)
        train = mod1.predict(X_train)
        test = mod1.predict(X_test)
        #Train.append(X_train)
        #Labels.append(y_train)
        #Train_test.append(X_test)
        #Labels_test.append(y_test)
        
        user_classifier = Sequential()
        user_classifier.add(Dense(8, input_shape=(nd,)))
        #user_classifier.add(Dense(8, activation='relu'))
        user_classifier.add(Dense(4, activation='softmax'))
        #user_emotion = user_classifier(features)
        #model = Model(input=row, output=user_emotion)
        user_classifier.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        user_classifier.fit(train, y_train,epochs=50,batch_size=8,validation_data=(test, y_test))
        print()
        print()
        print()
        print()
        print()

            #User_classifiers.append(user_classifier)
        #User_emotions.append(user_emotion)
        #Models.append(model)
        

In [ ]:
#,callbacks=[PlotLossesCallback()]
for i in range(len(Models)):
    Models[i].fit(Train[i], Labels[i],epochs=50,batch_size=8,validation_data=(Train_test[i], Labels_test[i]))
    print("done " + str(i))


In [ ]:
from sklearn.metrics import roc_auc_score,accuracy_score
import numpy as np
aucs = []
accs = []
for i in range(len(Models)):
    roc = 0
    
    y_true = np.argmax(Labels[i],axis = 1)
    len_tot = len(y_true)
    y_pred = Models[i].predict(Train[i])
    unique_elements, counts_elements = np.unique(y_true, return_counts=True)
    for x in unique_elements:
        if x == 0:
            
            y_pred0 = y_pred[:,0]
            y_true0 = np.copy(y_true)
            y_true0 [y_true0 == 0] = -1
            y_true0 [y_true0 != -1] = 0
            y_true0 [y_true0 == -1] = 1
            roc += (counts_elements[0]/len_tot)*roc_auc_score(y_true0,y_pred0)
            
            
        elif x == 1:
            
            y_pred1 = y_pred[:,1]
            y_true1 = np.copy(y_true)
            y_true1 [y_true1 != 1] = 0
            roc += (counts_elements[1]/len_tot)*roc_auc_score(y_true1,y_pred1)
            
        elif x == 2 :
            
            y_pred2 = y_pred[:,2]
            y_true2 = np.copy(y_true)
            y_true2 [y_true2 == 2] = -1
            y_true2 [y_true2 != -1] = 0
            y_true2 [y_true2 == -1] = 1
            roc += (counts_elements[2]/len_tot)*roc_auc_score(y_true2,y_pred2)
            
        else :
            
            y_pred3 = y_pred[:,3]
            y_true3 = np.copy(y_true)
            y_true3 [y_true3 == 3] = -1
            y_true3 [y_true3 != -1] = 0
            y_true3 [y_true3 == -1] = 1
            roc += (counts_elements[3]/len_tot)*roc_auc_score(y_true3,y_pred3)
            
    aucs.append(roc)
    accs.append(accuracy_score(y_true,np.argmax(y_pred,axis = 1)))
print(aucs)
print(accs)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
index = np.arange(len(aucs))
plt.bar(index, accs)
plt.xlabel('Users', fontsize=15)
plt.ylabel('AUC ROC score', fontsize=15)
plt.title('user emotion prediction using MTL')
plt.show()